# Day 3 - Conversational AI - aka Chatbot!

In [4]:
# imports

import os
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

In [5]:
# Load environment variables in a file called .env
# Print the key prefixes to help with any debugging

load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")

OpenAI API Key not set


In [6]:
# Initialize

openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

MODEL = 'llama3.2'

In [14]:
# Again, I'll be in scientist-mode and change this global during the lab
import os
import requests
import gradio as gr
from openai import OpenAI
from dotenv import load_dotenv
from IPython.display import Markdown, display

load_dotenv(override=True)

openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')
MODEL="llama3.2"

system_prompt = [
    {"role": "user", "content": "You are a helpful assistant"}]
user_prompt = [
    {"role": "user", "content": "Hi, there!"},
]

def message_gpt(prompt):
    response = openai.chat.completions.create(model=MODEL, messages=system_prompt)
    display(Markdown(response.choices[0].message.content))

message_gpt(user_prompt)

I'm here to assist you with any questions or tasks you may have. How can I help you today? Do you need information on a specific topic, need help with a problem, or perhaps want to discuss something? Let me know and I'll do my best to provide you with the assistance you require!

## And now, writing a new callback

We now need to write a function called:

`chat(message, history)`

Which will be a callback function we will give gradio.

### The job of this function

Take a message, take the prior conversation, and return the response.


In [15]:
def chat(message, history):
    return "bananas"

In [16]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


In [17]:
def chat(message, history):
    return f"You said {message} and the history is {history} but I still say bananas"

In [18]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7861
* To create a public link, set `share=True` in `launch()`.


## OK! Let's write a slightly better chat callback!

In [21]:

def chat(message, history):
    history = [{"role":h["role"], "content":h["content"]} for h in history]
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model="llama3.2", messages=messages)
    return response.choices[0].message.content


In [22]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7863
* To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "/Users/hadikhamsi/projects/llm_engineering/.venv/lib/python3.12/site-packages/gradio/queueing.py", line 759, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/hadikhamsi/projects/llm_engineering/.venv/lib/python3.12/site-packages/gradio/route_utils.py", line 354, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/hadikhamsi/projects/llm_engineering/.venv/lib/python3.12/site-packages/gradio/blocks.py", line 2116, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/hadikhamsi/projects/llm_engineering/.venv/lib/python3.12/site-packages/gradio/blocks.py", line 1621, in call_function
    prediction = await fn(*processed_input)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/hadikhamsi/projects/llm_engineering/.venv/lib

In [23]:
def chat(message, history):
    history = [{"role":h["role"], "content":h["content"]} for h in history]
    messages = [{"role": "system", "content": system_prompt}] + history + [{"role": "user", "content": message}]
    stream = openai.chat.completions.create(model="llama3.2", messages=messages, stream=True)
    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

In [24]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7864
* To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "/Users/hadikhamsi/projects/llm_engineering/.venv/lib/python3.12/site-packages/gradio/queueing.py", line 759, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/hadikhamsi/projects/llm_engineering/.venv/lib/python3.12/site-packages/gradio/route_utils.py", line 354, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/hadikhamsi/projects/llm_engineering/.venv/lib/python3.12/site-packages/gradio/blocks.py", line 2116, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/hadikhamsi/projects/llm_engineering/.venv/lib/python3.12/site-packages/gradio/blocks.py", line 1635, in call_function
    prediction = await utils.async_iteration(iterator)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/hadikhamsi/projects/llm

## OK let's keep going!

Using a system message to add context, and to give an example answer.. this is "one shot prompting" again

In [25]:
system_message = "You are a helpful assistant in a clothes store. You should try to gently encourage \
the customer to try items that are on sale. Hats are 60% off, and most other items are 50% off. \
For example, if the customer says 'I'm looking to buy a hat', \
you could reply something like, 'Wonderful - we have lots of hats - including several that are part of our sales event.'\
Encourage the customer to buy hats if they are unsure what to get."

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch()

In [ ]:
system_message += "\nIf the customer asks for shoes, you should respond that shoes are not on sale today, \
but remind the customer to look at hats!"

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch()

In [ ]:

def chat(message, history):
    history = [{"role":h["role"], "content":h["content"]} for h in history]
    relevant_system_message = system_message
    if 'belt' in message.lower():
        relevant_system_message += " The store does not sell belts; if you are asked for belts, be sure to point out other items on sale."
    
    messages = [{"role": "system", "content": relevant_system_message}] + history + [{"role": "user", "content": message}]

    stream = openai.chat.completions.create(model=MODEL, messages=messages, stream=True)

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        yield response

In [ ]:
gr.ChatInterface(fn=chat, type="messages").launch()

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../assets/business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business Applications</h2>
            <span style="color:#181;">Conversational Assistants are of course a hugely common use case for Gen AI, and the latest frontier models are remarkably good at nuanced conversation. And Gradio makes it easy to have a user interface. Another crucial skill we covered is how to use prompting to provide context, information and examples.
<br/><br/>
Consider how you could apply an AI Assistant to your business, and make yourself a prototype. Use the system prompt to give context on your business, and set the tone for the LLM.</span>
        </td>
    </tr>
</table>